In [ ]:
### 중요: lOSS를 계산 ###
# 손실 계산 (CrossEntropyLoss)
# PyTorch의 CrossEntropyLoss는 입력을 (N, C) 형태로 받기를 원한다.
# N: 전체 샘플 수 (여기서는 Batch_size * Sequence_length)
# C: 클래스 수 (여기서는 Vocab_size)
# 따라서 3차원 텐서를 2차원으로 평탄화(flatten) 해야 한다.
# logits shape: (batch_size, sequence_length, vocab_size) 3차원을
# logits.flatten(0, 1) -> (batch_size * sequence_length, vocab_size) 으로 2차원으로 변환
# target_batch.flatten() -> (batch_size * sequence_length)
loss = torch.nn.functional.cross_entropy(
    logits.flatten(0, 1), 
    target_batch.flatten()
)
# CrossEntropyLoss는 (N, C) 형태의 입력과 (N,) 형태의 정답 레이블을 요구

In [ ]:
""" 모델 학습 과정 (반드시 외우기)
    # model.train() 반드시 필요 // evaluation이나 test에서는 model.eval()
    # 1. 기울기 초기화 > .zero_grad()
    # 2. 데이터를 GPU로 이동 > .to(device)
    # 3. 모델의 순전파 > model(...)
    # 4. 손실 계산 > flatten(...)
    # 5. 역전파 > .backward()
    # 6. 가중치 업데이트 > step()
"""
    
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):

    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen = 0
    global_step = -1
    
     for epoch in range(num_epochs):
        model.train()  # 훈련 시작 전 반드시 train 모드 설정
        
        for input_batch, target_batch in train_loader:
            # 1. 이전 배치에서 계산된 기울기 초기화(필수)
            optimizer.zero_grad() 
            
            # 2. 데이터를 GPU로 이동
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)
            
            # 3. 모델의 순전파(Forward Pass)
            logits = model(input_batch)
            
            # 4. 손실 계산
            loss = torch.nn.functional.cross_entropy(
                input_batch.flatten(0,1),
                target_batch.flatten(),
            )
            
            # 5. 역전파 (Backpropagation): 각 파라미터별 기울기(Gradient) 계산
            loss.backward()
            
            # 6. 가중치 업데이트: 계산된 기울기를 이용해 파라미터 수정
            optimizer.step()
            
            tokens_seen += input_batch.numel() # 처리한 토큰 수 카운트
            global_step += 1
            
    

In [ ]:
### 단계별 포인트

def main(gpt_config, settings):
    ##############################
    # 1. 데이터 준비
    ##############################
    
    ##############################
    # 2. 모델 및 옵티마이저 초기화
    ##############################
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=settings["learning_rate"], weight_decay=settings["weight_decay"]
    ) # 이걸 다 외워야 할건 아닌거 같고 그냥 이런식으로 optimizer 선언한다고 숙지하면 될 듯.
    
    ##############################
    # 3. 데이터 로더 구축 : 어떤식으로 데이터 나누는지 확인
    ##############################
    train_ratio = 0.90
    split_idx = int(train_ratio * len(text_data))
    
    # 훈련 데이터 로더: 순서를 섞음(Shuffle=True)
    train_loader = create_dataloader_v1(
        text_data[:split_idx], # 0~ 전체 0.9에 대항하는 데이터까지
        ...
        drop_last=True, # 뒤에 자투리 남으면 버리자.
        shuffle=True, # 데이터 섞어라. 학습에는 중요
        ...
    )
    
    # 검증 데이터 로더: 순서를 섞지 않음(Shuffle=False) -> 평가는 일관되게
    val_loader = create_dataloader_v1(
        
        text_data[split_idx:], # 0.9 이후의 것들로 테스트
        ...
        drop_last=False, # 버리지 마라
        shuffle=False,   # 섞지마!!
        ...
    )
    ##############################
    # 4. 훈련 시작
    ##############################